### PydanticOutputParser를 활용한 게임 추천

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("OutputParser_Practice")

LangSmith 추적을 시작합니다.
[프로젝트명]
OutputParser_Practice


In [8]:
import os
from typing import List, Optional

from pydantic import BaseModel, Field, ValidationError

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain_core.pydantic_v1 import BaseModel as V1BaseModel  # Pydantic v1 호환성
from langchain_core.pydantic_v1 import Field as V1Field, validator as V1Validator # Pydantic v1 호환성


c:\Users\SBA\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-Us6BDj1P-py3.11\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
# 1. Pydantic 모델 정의
# 지원 플랫폼과 출처 정보를 포함한 최종 모델
class GameRecommendation(BaseModel):
    title: str = Field(description="추천 게임의 제목")
    genre: List[str] = Field(description="게임 장르 목록 (예: RPG, Action, Indie)")
    summary: str = Field(description="게임에 대한 간략한 설명")
    similarity: str = Field(description="해본 게임과의 유사성")
    developer: str = Field(description="개발사 또는 제작사")
    platforms: List[str] = Field(description="지원 플랫폼 목록 (예: PC, PlayStation, Nintendo Switch, Mobile)")
    source: Optional[str] = Field(description="추천 정보의 출처 URL. 찾을 수 없는 경우 빈 문자열")

In [ ]:
# 2. 파서 및 LLM 설정
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# PydanticOutputParser를 사용해 모델과 연결
pydantic_parser = PydanticOutputParser(pydantic_object=GameRecommendation)

# OutputFixingParser를 추가하여 출력 오류에 대비
parser = OutputFixingParser.from_llm(parser=pydantic_parser, llm=llm)

In [ ]:
# 3. 프롬프트 템플릿 정의
prompt = PromptTemplate(
    template="""너는 전문 게임 추천가야. 다음 요청에 따라 사용자에게 게임을 추천해줘.
    요청: {request}

    {format_instructions}
    
    주의사항:
    - 요청에 맞는 게임을 딱 하나만 추천해줘.
    -오직 신뢰할 수 있는 정보를 바탕으로 추천해줘.
    - Platforms 필드에는 'PC (Steam)', 'PC (Epic Games Store)', 'PlayStation 5', 'Xbox Series X/S', 'Nintendo Switch', 'Mobile' 등 구체적인 플랫폼을 포함해줘.
    - Source 필드는 웹사이트 URL 형식으로 작성해줘. 출처를 찾지 못하면 None으로 반환해도 돼.
    """,
    input_variables=["request"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [14]:
# 4. 체인 연결 및 실행
chain = prompt | llm | parser

# 사용자 요청
user_request = "내가 해본 게임은 '카트라이더'야. 이 게임과 비슷한 장르의 게임을 추천해줘."

try:
    # 체인을 실행하여 결과 얻기
    recommendation = chain.invoke({"request": user_request})

    # Pydantic 모델 인스턴스 출력
    print("=== 게임 추천 결과 ===")
    print(f"제목: {recommendation.title}")
    print(f"장르: {', '.join(recommendation.genre)}")
    print(f"요약: {recommendation.summary}")
    print(f"유사성: {recommendation.similarity}")
    print(f"개발사: {recommendation.developer}")
    print(f"플랫폼: {', '.join(recommendation.platforms)}")
    print(f"출처: {recommendation.source}")

except ValidationError as e:
    print("유효성 검사 오류:", e)
    # OutputFixingParser를 사용했기 때문에 이 오류는 발생할 가능성이 낮아집니다.


=== 게임 추천 결과 ===
제목: Mario Kart 8 Deluxe
장르: Racing, Party
요약: Mario Kart 8 Deluxe는 다양한 캐릭터와 트랙에서 경주를 즐길 수 있는 인기 카트 레이싱 게임입니다. 친구들과 함께 멀티플레이어 모드로 즐길 수 있으며, 다양한 아이템과 커스터마이징 옵션이 특징입니다.
유사성: 카트라이더와 유사하게 경주와 아이템 사용이 중요한 레이싱 게임으로, 캐주얼한 플레이가 가능합니다.
개발사: Nintendo
플랫폼: Nintendo Switch
출처: https://www.nintendo.com/games/detail/mario-kart-8-deluxe-switch/


### 문장 난이도 분류

In [15]:
import os
from enum import Enum

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.output_parsers import EnumOutputParser

In [23]:
# 1. 난이도 Enum 타입 정의
# Enum 클래스는 str을 상속받아 파싱을 더 쉽게 만듭니다.
class Difficulty(str, Enum):
    VERY_EASY = "매우_쉬움"
    EASY = "쉬움"
    NORMAL = "보통"
    HARD = "어려움"
    VERY_HARD = "매우_어려움"

In [18]:
# 2. LLM 및 파서 설정
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# EnumOutputParser 설정
enum_parser = EnumOutputParser(enum=Difficulty)

In [26]:
# 3. 프롬프트 템플릿 정의
# 프롬프트에 EnumOutputParser가 생성한 포맷 지시문을 포함시킵니다.
prompt = PromptTemplate(
    template="""주어진 문장의 난이도를 분석하고, 아래 형식에 맞춰 분류해줘.
    
    문장: {sentence}

    {format_instructions}

    주의사항:
    - 오직 분류된 난이도만 출력하고, 다른 설명이나 부가적인 내용은 일절 포함하지 마.
    - 예를 들어, '매우_어려움'만 출력하고 '난이도: 매우_어려움'과 같은 형태도 사용하지 마.
    """,
    input_variables=["sentence"],
    partial_variables={"format_instructions": enum_parser.get_format_instructions()},
)

In [27]:
# 4. 체인 연결 및 실행
chain = prompt | llm | enum_parser

In [28]:
# 다양한 난이도의 문장들
sentences = [
    "나는 학생입니다.",  # 매우_쉬움
    "그는 운동선수로서 꾸준히 훈련에 매진했다.", # 쉬움
    "인간의 의식은 뇌의 복잡한 신경 회로망에서 비롯된 현상이다.", # 보통
    "가역적이고 자기 조직적인 시스템의 비선형 동역학은 혼돈 이론에 의해 설명될 수 있다.", # 어려움
    "다양한 매체에서 수렴적 진화론과 관련한 오개념이 확산되어, 생물학계가 우려를 표하고 있다." # 어려움 또는 매우_어려움
]

In [29]:
print("=== 문장 난이도 분류 결과 ===")
for i, sentence in enumerate(sentences):
    try:
        # 체인 실행
        result = chain.invoke({"sentence": sentence})
        print(f"[{i+1}] 문장: '{sentence}'")
        print(f"    -> 난이도: {result.value}\n")

    except Exception as e:
        print(f"[{i+1}] 문장: '{sentence}'")
        print(f"    -> 오류 발생: {e}\n")

=== 문장 난이도 분류 결과 ===
[1] 문장: '나는 학생입니다.'
    -> 난이도: 매우_쉬움

[2] 문장: '그는 운동선수로서 꾸준히 훈련에 매진했다.'
    -> 난이도: 쉬움

[3] 문장: '인간의 의식은 뇌의 복잡한 신경 회로망에서 비롯된 현상이다.'
    -> 난이도: 어려움

[4] 문장: '가역적이고 자기 조직적인 시스템의 비선형 동역학은 혼돈 이론에 의해 설명될 수 있다.'
    -> 난이도: 어려움

[5] 문장: '다양한 매체에서 수렴적 진화론과 관련한 오개념이 확산되어, 생물학계가 우려를 표하고 있다.'
    -> 난이도: 어려움



##### 일단 출력은 됐으나 문장의 난이도는 사람마다 다르게 느낄 수 있기 때문에 AI가 보편적인 기준으로 판단하는 데 한계가 있습니다.